In [ ]:
#   Ejercicio 2

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query2 = proveedores.find(
    {"razon_social": {"$regex": "Tecnología", "$options": "i"}},
    {"_id": 0, "CUIT": 1, "telefonos": 1}
)

for p in query2:
    print(p)

save_to_txt("query2.txt", list(query2))
      

In [ ]:
#   Ejercicio 3

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query3 = proveedores.aggregate([
    {"$unwind": "$telefonos"},
    {"$project": {
        "_id": 0,
        "razon_social": 1,
        "CUIT": 1,
        "telefono": "$telefonos"
    }}
])
save_to_txt("query3.txt", list(query3))

In [11]:
#   Ejercicio 5

from neo4j import GraphDatabase
import json

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, ensure_ascii=False, indent=2) + "\n")

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def obtener_proveedores_sin_ordenes(tx):
    result = tx.run("""
        MATCH (p:Proveedor) 
        WHERE NOT ((:Pedido)-[:ES_DE]->(p))
        RETURN p.id_proveedor AS id_proveedor,
               p.CUIT AS CUIT_proveedor,
               p.razon_social AS razon_social,
               p.tipo_sociedad AS tipo_sociedad,
               p.direccion AS direccion,
               p.activo AS activo,
               p.habilitado AS habilitado
    """)
    return [
        {
            "id_proveedor": record["id_proveedor"],
            "CUIT_proveedor": record["CUIT_proveedor"],
            "razon_social": record["razon_social"],
            "tipo_sociedad": record["tipo_sociedad"],
            "direccion": record["direccion"],
            "activo": record["activo"],
            "habilitado": record["habilitado"]
        } 
        for record in result
    ]


with driver.session() as session:
    query5 = session.execute_read(obtener_proveedores_sin_ordenes)

save_to_txt("query5.txt", query5)

print(f"✅ Se guardaron {len(query5)} proveedores sin órdenes en 'query5.txt'.")

✅ Se guardaron 2 proveedores sin órdenes en 'query5.txt'.


In [ ]:
#   Ejercicio 6

from pymongo import MongoClient
import json


client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query6 = proveedores.aggregate([
    {"$lookup": {
        "from": "pedidos",
        "localField": "_id",
        "foreignField": "id_proveedor",
        "as": "ordenes"
    }},
    {"$project": {
        "_id": 0,
        "id_proveedor": "$_id",
        "razon_social": 1,
        "total_ordenes": {"$size": "$ordenes"},
        "monto_total_sin_iva": {
            "$sum": {"$map": {
                "input": "$ordenes",
                "as": "orden",
                "in": "$$orden.total_sin_iva"
            }}
        },
        "monto_total_con_iva": {
            "$sum": {
                "$map": {
                    "input": "$ordenes",
                    "as": "orden",
                    "in": {
                        "$add": ["$$orden.total_sin_iva", "$$orden.iva"]
                    }
                }
            }
        }
    }}
])
save_to_txt("query6.txt", list(query6))

In [8]:
#   Ejercicio 8

from neo4j import GraphDatabase
import json

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, ensure_ascii=False, indent=2) + "\n")

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def obtener_productos_pedidos_al_menos_una_vez(tx):
    result = tx.run("""
        MATCH (:Pedido)-[:INCLUYE]->(p:Producto)
        RETURN DISTINCT p.id_producto AS id_producto,
               p.descripcion AS descripcion,
               p.marca AS marca,
               p.categoria AS categoria,
               p.precio AS precio,
               p.stock_actual AS stock_actual,
               p.stock_futuro AS stock_futuro
    """)
    return [
        {
            "id_producto": record["id_producto"],
            "descripcion": record["descripcion"],
            "marca": record["marca"],
            "categoria": record["categoria"],
            "precio": record["precio"],
            "stock_actual": record["stock_actual"],
            "stock_futuro": record["stock_futuro"]
        }
        for record in result
    ]

with driver.session() as session:
    query8 = session.execute_read(obtener_productos_pedidos_al_menos_una_vez)

save_to_txt("query8.txt", query8)

In [ ]:
#   Ejercicio 9

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
pedidos = db["pedidos"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

query9 = pedidos.aggregate([
    {"$unwind": "$detalle"},
    {"$lookup": {
        "from": "productos",
        "localField": "detalle.id_producto",
        "foreignField": "_id",
        "as": "producto"
    }},
    {"$unwind": "$producto"},
    {"$match": {
        "producto.marca": "COTO"
    }},
    {"$project": {
        "_id": 1,
        "fecha": 1,
        "id_proveedor": 1,
        "total_sin_iva": 1,
        "iva": 1,
        "detalle": 1,
        "producto": 1
    }}
])
save_to_txt("query9.txt", list(query9))


In [10]:
#   Ejercicio 11

from neo4j import GraphDatabase
import json

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, ensure_ascii=False, indent=2) + '\n')

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def obtener_productos_no_pedidos(tx):
    result = tx.run("""
        MATCH (p:Producto)
        WHERE NOT EXISTS((:Pedido)-[:INCLUYE]->(p))
        RETURN p.id_producto AS id_producto,
               p.descripcion AS descripcion,
               p.marca AS marca,
               p.categoria AS categoria,
               p.precio AS precio,
               p.stock_actual AS stock_actual,
               p.stock_futuro AS stock_futuro
    """)
    return [
        {
            "id_producto": record["id_producto"],
            "descripcion": record["descripcion"],
            "marca": record["marca"],
            "categoria": record["categoria"],
            "precio": record["precio"],
            "stock_actual": record["stock_actual"],
            "stock_futuro": record["stock_futuro"]
        } for record in result
    ]


with driver.session() as session:
    productos_no_pedidos = session.execute_read(obtener_productos_no_pedidos)

save_to_txt("query11.txt", productos_no_pedidos)

In [ ]:
#   Ejercicio 12

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db.command({
    "create": "proveedores_inhabilitados_activos",
    "viewOn": "proveedores",
    "pipeline": [
        {"$match": {
            "activo": True,
            "habilitado": False
        }},
        {"$project": {
            "_id": 0,
            "id_proveedor": "$_id",
            "razon_social": 1,
            "CUIT": 1,
            "direccion": 1,
            "tipo_sociedad": 1
        }}
    ]
})


## info de la vista en query12.txt
vista = db["proveedores_inhabilitados_activos"]
rta = list(vista.find())
save_to_txt("query12.txt", rta)


In [ ]:
#   Ejercicio 14


In [ ]:
#   Ejercicio 15

from pymongo import MongoClient
from datetime import datetime
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
proveedores = db["proveedores"]

## data de la orden 
nueva_orden = {
    "_id": 364,
    "id_proveedor": 1,
    "fecha": datetime.strptime("2025-06-19", "%Y-%m-%d"),
    "total_sin_iva": 1000.0,
    "iva": 210.0,
    "detalle": [
        {"id_producto": 1, "nro_item": 1, "cantidad": 3}
    ]
}

if proveedores.find_one({"_id": nueva_orden["id_proveedor"]}):
    pedidos.insert_one(nueva_orden)
    with open("query15.txt", "w") as f:
        f.write("Nueva orden registrada:\n")
        f.write(json.dumps(nueva_orden, default=str, indent=2))
else:
    with open("query15.txt", "w") as f:
        f.write(f"No se pudo registrar orden: proveedor con ID {nueva_orden['id_proveedor']} no existe.\n")